In [25]:
from keras import backend as K
import time
import matplotlib.pyplot as plt
import numpy as np_utils
%matplotlib inline
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, DepthwiseConv2D, Conv2D, SeparableConv2D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, concatenate, LeakyReLU
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l2
%config InlineBackend.figure_format='retina'
from keras_contrib.callbacks import CyclicLR
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from data_science_utils.vision.keras import *
from time import time
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno as msno
import re
from joblib import Parallel, delayed
from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils
from data_science_utils import misc as misc
from data_science_utils import preprocessing as pp_utils
from data_science_utils import nlp as nlp_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from data_science_utils.dataframe import get_specific_cols

import more_itertools
from more_itertools import flatten
import ast
from sklearn.preprocessing import LabelEncoder

import gc

In [2]:
import sys
import os
sys.path.append(os.getcwd())
from importlib import reload
import lib
reload(lib)
from lib import *

from oclr import OneCycleLR, LRFinder

<module 'lib' from '/home/ec2-user/SageMaker/ML_hackathon_2019/lib.py'>

In [3]:
df_train = pd.read_csv("price_prediction/train.csv")
df_test = pd.read_csv("price_prediction/test.csv")

In [4]:
df_train['text'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_train['text']))
df_train['text_encoded'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_train['text_encoded']))
df_train['char_encoded'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_train['char_encoded']))

df_test['text'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_test['text']))
df_test['text_encoded'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_test['text_encoded']))
df_test['char_encoded'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_test['char_encoded']))


# GL encodings

In [5]:
from data_science_utils.preprocessing import NeuralCategoricalFeatureTransformer

ct_nn = NeuralCategoricalFeatureTransformer(cols=["GL"],prefix="gl_encoded_",
                                            target_columns=["PRICE"],verbose=0,n_components=16,n_iter=200,)

ct_nn.fit(df_train)

ct_nn.skip_fit = True

Neural Categorical fit start at: 2019-06-13 20:13:23.712711
Instructions for updating:
Colocations handled automatically by placer.
Shape of Input to Neural Network: (61, 61), Output shape: (61, 69)
Instructions for updating:
Use tf.cast instead.


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Neural Categorical fit done at: 2019-06-13 20:13:41.422190


In [6]:
df_train = ct_nn.transform(df_train)
df_test = ct_nn.transform(df_test)

In [7]:
gl_cols = get_specific_cols(df_train,prefix='gl_')

In [11]:
start = time()
glove = api.load("glove-twitter-25") 
print("total = ",(time()-start))

ptr_glove = PreTrainedEmbeddingsTransformer(glove,size=25)
ptr_glove.fit()

df_test['glove_encoded'] = ptr_glove.transform(df_test['text'].values)
df_train['glove_encoded'] = ptr_glove.transform(df_train['text'].values)


[==================================================] 100.0% 104.8/104.8MB downloaded


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


total =  55.031386852264404


In [8]:
start = time()
glove = api.load("glove-twitter-50") 
print("total = ",(time()-start))

ptr_glove_50 = PreTrainedEmbeddingsTransformer(glove,size=50)
ptr_glove_50.fit()

df_test['glove_encoded-50'] = ptr_glove_50.transform(df_test['text'].values)
df_train['glove_encoded-50'] = ptr_glove_50.transform(df_train['text'].values)


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


total =  74.3492660522461
Fasttext Transforms start at: 2019-06-13 20:15:43.567292
Number of Unique Test Tokens for Fasttext transform 160522



Fasttext Transforms done at: 2019-06-13 20:16:00.514933
Fasttext Transforms start at: 2019-06-13 20:16:00.533301
Number of Unique Test Tokens for Fasttext transform 332242



Fasttext Transforms done at: 2019-06-13 20:16:34.420065


In [ ]:
start = time()
fasttext = api.load("fasttext-wiki-news-subwords-300") 
print("total = ",(time()-start))

ptr = PreTrainedEmbeddingsTransformer(fasttext,size=300)
ptr.fit()

df_train['fasttext_encoded'] = ptr.transform(df_train['text'].values)
df_test['fasttext_encoded'] = ptr.transform(df_test['text'].values)

In [23]:
batch_size = 256
epochs = 5

max_char_features = 128
char_maxlen = 500
char_embedding_dims = 20

text_max_features = 50000
text_maxlen = 100
text_embedding_dims = 25

enc_maxlen = 100
enc_embedding_dims = 50

In [10]:



X_enc,X_text,X_char,X_gl,y = df_train['glove_encoded-50'].values,df_train['text_encoded'].values,df_train['char_encoded'].values,df_train[gl_cols],df_train['PRICE'].values
x_train_enc, x_test_enc,x_train_text, x_test_text, x_train_char, x_test_char,x_gl_train,x_gl_test, y_train, y_test = train_test_split(X_enc,X_text,X_char,X_gl, y, test_size=0.2, random_state=42)

x_train_enc = sequence.pad_sequences(x_train_enc, maxlen=enc_maxlen)
x_test_enc = sequence.pad_sequences(x_test_enc, maxlen=enc_maxlen)

x_train_text = sequence.pad_sequences(x_train_text, maxlen=text_maxlen)
x_test_text = sequence.pad_sequences(x_test_text, maxlen=text_maxlen)

x_train_char = sequence.pad_sequences(x_train_char, maxlen=char_maxlen)
x_test_char = sequence.pad_sequences(x_test_char, maxlen=char_maxlen)

x_train_enc = x_train_enc.reshape((-1,100,enc_embedding_dims,1))
x_test_enc = x_test_enc.reshape((-1,100,enc_embedding_dims,1))

datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,)
datagen.fit(x_train_enc)
train_iterator = datagen.flow(x_train_enc, y_train, batch_size=len(x_train_enc),shuffle=True)
validation_iterator = datagen.flow(x_test_enc, y_test, batch_size=len(x_test_enc),shuffle=True)

x_train_enc = train_iterator.next()[0]
x_test_enc = validation_iterator.next()[0]

x_train_enc = x_train_enc.reshape((-1,100,enc_embedding_dims))
x_test_enc = x_test_enc.reshape((-1,100,enc_embedding_dims))




# Helper

In [11]:
def intermediate_joiner(intermediates):
    x = concatenate(intermediates)
    x = transition_layer(x, n_kernels=64,dropout=0)
    x1 = conv_layer(x,n_kernels=64,kernel_size=3,padding='same')
    x2 = conv_layer(x1,n_kernels=128,kernel_size=3,padding='same')
    x = concatenate([x,x1,x2])
    x = transition_layer(x, n_kernels=64, dropout=0)
    
    x1 = AveragePooling1D()(x) 
    x2 = MaxPooling1D()(x)
    x = concatenate([x1,x2])
    x = transition_layer(x, n_kernels=64, dropout=0)
    x = conv_layer(x,n_kernels=128,kernel_size=3,padding='same')
    
    print("Before FC Intermediate =",K.int_shape(x))
    out1 = GlobalAveragePooling1D()(x)
    out2 = GlobalMaxPooling1D()(x)
    out = concatenate([out1,out2])
    
    return out


In [26]:
def aux_output_fcnn(inputs):
    x = Dense(16)(inputs)
    x = LeakyReLU()(x)
    x = Dropout(0.1)(x)
    
    x = Dense(1)(x)
    return x

In [13]:
def word_cnn(inputs):
    x = inputs
    filters = 250
    kernel_size = 3
    x = Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(x)

    x1 = MaxPooling1D(pool_size=2)(x)
    x2 = AveragePooling1D(pool_size=2)(x)
    x = concatenate([x1,x2])

    xp = x
    x = transition_layer(x, n_kernels=32,dropout=0)
    x = conv_layer(x, n_kernels=64,kernel_size=3,dilation_rate=1,padding='same')
    x = concatenate([x,xp])
    x = transition_layer(x, n_kernels=128,dropout=0)
    intermidate = x
    # we use max pooling:
    x2 = GlobalAveragePooling1D()(x)
    x = GlobalMaxPooling1D()(x)
    x = concatenate([x,x2])
    
    print("W1 Intermediate =",K.int_shape(intermidate))
    return intermidate, aux_output_fcnn(x)

In [14]:
def word_cnn_v2(inputs):
    
    filters = 250
    kernel_size = 3
    x = inputs
    x = Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1)(x)
    
    x1 = MaxPooling1D(pool_size=2)(x)
    x2 = AveragePooling1D(pool_size=2)(x)
    x = concatenate([x1,x2])
    
    xp = x
    x = transition_layer(x, n_kernels=64,dropout=0)
    xp2 = conv_layer(x,n_kernels=64,kernel_size=3,dilation_rate=1,padding='same')

    x = concatenate([xp2, xp])
    x = transition_layer(x, n_kernels=64,dropout=0)
    xp3 = conv_layer(x,n_kernels=64,kernel_size=3,dilation_rate=2,padding='same')

    x = concatenate([xp,xp2,xp3])
    x = transition_layer(x, n_kernels=128,dropout=0)
    intermidate = x
    # we use max pooling:
    x2 = GlobalAveragePooling1D()(x)
    x1 = GlobalMaxPooling1D()(x)
    x = concatenate([x1,x2])
    print("W2 Intermediate =",K.int_shape(intermidate))
    return intermidate, aux_output_fcnn(x)

In [15]:
def char_cnn():
    main_input = Input(shape=(char_maxlen,), dtype='int32',)
    x = Embedding(output_dim=char_embedding_dims, input_dim=max_char_features, input_length=char_maxlen)(main_input)
    
    x1 = MaxPooling1D(pool_size=5)(x)
    x2 = AveragePooling1D(pool_size=5)(x)
    x = concatenate([x1,x2])
    
    x = conv_layer(x,n_kernels=64, kernel_size=3,padding='valid')
    
    x1 = MaxPooling1D(pool_size=2)(x)
    x2 = AveragePooling1D(pool_size=2)(x)
    x = concatenate([x1,x2])
    
    xp1 = x
    
    x = transition_layer(x, n_kernels=32, dropout=0)
    x = conv_layer(x,n_kernels=64,kernel_size=15,padding='same')
    xp2 = x
    x = concatenate([x,xp1])
    
    x = transition_layer(x, n_kernels=32, dropout=0)
    x = conv_layer(x,n_kernels=64,kernel_size=15,padding='same', dilation_rate=2)
    x = concatenate([x,xp2])
    intermidate = x
    x2 = GlobalAveragePooling1D()(x)
    x1 = GlobalMaxPooling1D()(x)
    x = concatenate([x1,x2])
    print("Char Cnn Intermediate =",K.int_shape(intermidate))
    return main_input, intermidate, aux_output_fcnn(x)


In [16]:
def pretrained_embedding_cnn():
    main_input = Input(shape=(enc_maxlen,enc_embedding_dims), dtype='float32')
    x = main_input
    x = conv_layer(x,n_kernels=64,kernel_size=3,padding='valid')
    
    x1 = MaxPooling1D(pool_size=2)(x)
    x2 = AveragePooling1D(pool_size=2)(x)
    x = concatenate([x1,x2])
    
    xp = x
    x = transition_layer(x, n_kernels=32,dropout=0)
    x = conv_layer(x,n_kernels=64,kernel_size=3,padding='same')
    x = transition_layer(x, n_kernels=32,dropout=0)

    xp2 = x
    x = concatenate([x,xp])
    x = transition_layer(x, n_kernels=32,dropout=0)
    x = conv_layer(x,n_kernels=64,kernel_size=3,dilation_rate=2,padding='same')
    x = transition_layer(x, n_kernels=32,dropout=0)

    x = concatenate([x,xp,xp2])

    x = transition_layer(x, n_kernels=128,dropout=0)
    intermidate = x
    print("Pretrained Intermediate =",K.int_shape(intermidate))
    x = pre_dense_layer(x)
    return main_input,intermidate, aux_output_fcnn(x)

In [27]:
def final_layer(flattened,aux):
    
    x = concatenate([flattened,aux])

    # We add a vanilla hidden layer:
    x = Dense(64)(x)
    x = Dropout(0.1)(x)
    x = LeakyReLU()(x)

    # We project onto a single unit output layer, and squash it with a sigmoid:
    output = Dense(1)(x)
    return output

# Ensemble

In [28]:
word_input = Input(shape=(text_maxlen,), dtype='int32')
x = Embedding(text_max_features,
            text_embedding_dims,
            input_length=text_maxlen)(word_input)

w1_intermidate, w1_output = word_cnn(x)

c_inputs,c_intermidate, c_output = char_cnn()
p_inputs,p_intermidate, p_output = pretrained_embedding_cnn()


auxiliary_input =  Input(shape=(len(gl_cols),), dtype='float32', name='aux_input')
x_aux = Dense(4)(auxiliary_input)

intermediates = [w1_intermidate,c_intermidate,p_intermidate]
x = intermediate_joiner(intermediates)

op = final_layer(x,x_aux)

outputs = [w1_output,c_output,p_output,op]
inputs = [word_input,c_inputs,p_inputs,auxiliary_input]

model = Model(inputs=inputs, outputs=outputs)

optimizer = Adam(lr=0.0001,)
model.compile(loss='mean_squared_error',
              optimizer=optimizer,
              metrics=['mae'],loss_weights=[0.3,0.3,0.3,1.0])

print("Params = ",model.count_params())
# 1634125

W1 Intermediate = (None, 49, 128)
Char Cnn Intermediate = (None, 49, 128)
Pretrained Intermediate = (None, 49, 128)
Before FC Intermediate = (None, 24, 128)
Params =  1652048


In [ ]:
model.fit([x_train_text,x_train_char,x_train_enc,x_gl_train], [y_train,y_train,y_train,y_train],
          batch_size=batch_size,
          epochs=epochs,
          validation_data=([x_test_text,x_test_char,x_test_enc,x_gl_test], [y_test,y_test,y_test,y_test]),
         shuffle=True)

Train on 1159686 samples, validate on 289922 samples
Epoch 1/5
1159686/1159686 [==============================] - 666s 574us/step - loss: 7666831663.5923 - dense_33_loss: 4035421708.7536 - dense_35_loss: 4036365429.8985 - dense_37_loss: 4036491918.9434 - dense_40_loss: 4034347805.8960 - dense_33_mean_absolute_error: 885.8146 - dense_35_mean_absolute_error: 959.4911 - dense_37_mean_absolute_error: 953.7178 - dense_40_mean_absolute_error: 872.0405 - val_loss: 33061327.8092 - val_dense_33_loss: 17309083.3915 - val_dense_35_loss: 19268256.5459 - val_dense_37_loss: 19497984.7625 - val_dense_40_loss: 16238730.1663 - val_dense_33_mean_absolute_error: 822.4257 - val_dense_35_mean_absolute_error: 879.6698 - val_dense_37_mean_absolute_error: 858.1819 - val_dense_40_mean_absolute_error: 916.6244
Epoch 2/5
1159686/1159686 [==============================] - 652s 562us/step - loss: 7663575750.5892 - dense_33_loss: 4033130301.4367 - dense_35_loss: 4035736284.7837 - dense_37_loss: 4036085803.0274 - de

In [ ]:
1+1